# <a id='toc1_'></a>[Using Bria 1.4 with AWS JumpStart](#toc0_)

This sample notebook shows you how to deploy BRIA v1.4 Safe for Commercial Use Model as an endpoint on Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.
 
<a id='toc1_1_'></a>[Prerequisites](#toc0_)

1. **Note**: Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.

1. Ensure that IAM role used has **AmazonSageMakerFullAccess**

1. To deploy the ML model successfully using the steps in this notebook, ensure that either:
    1. Your IAM role has the following three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. Or your AWS account has a subscription to [the Bria 1.4 - PLACEHOLDER MARKETPLACE LINK](https://aws.amazon.com/marketplace/pp/) . If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)


## <a id='toc1_3_'></a>[Usage instructions](#toc0_)
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).


   
- [1. Subscribe to the Bria Model Package](#toc3_)    
- [2: Create an endpoint and perform real-time inference](#toc4_)    
  - [A: Text to image](#toc4_1_)    
- [3: Delete the endpoint](#toc5_)    

# <a id='toc4_'></a>[2: Create an endpoint and perform real-time inference](#toc0_)

In [ ]:
from sagemaker.utils import name_from_base

endpoint_name = name_from_base('bria-1-5-jumpstart')

Once you have subscribed to Bria, get the Model Package ARN using the map below:


In [ ]:

model_package_map = {
    "us-east-1": "xx",
    "us-east-2": "xx",
    "us-west-2": "xx",
    "ca-central-1": "xx",
    "eu-central-1": "xx",
    "eu-west-1": "xx",
    "eu-west-2": "xx",
    "ap-northeast-2": "xx",
    "ap-northeast-1": "xx",
    "ap-south-1": "xx"
}


region = boto3.Session().region_name
if region not in model_package_map.keys():
    raise ("UNSUPPORTED REGION")
package_arn = model_package_map[region]

role_arn = get_execution_role()
sagemaker_session = sagemaker.Session()

Create a deployable `ModelPackage`. For Bria 1.5 use one of the following instances types: ml.g5.2xlarge, ml.p4d.24xlarge, p4de.24xlarge. Specify it as `instance_type` below.


In [ ]:
model = ModelPackage(role=role_arn,model_package_arn=package_arn,sagemaker_session=sagemaker_session,predictor_cls=BriaPredictor)


# Deploy the ModelPackage. This will take 5-10 minutes to run

instance_type="ml.g5.2xlarge" # valid instance types for this model are ml.g5.2xlarge, p4d.24xlarge, and p4de.24xlarge
deployed_model = model.deploy(initial_instance_count=1,instance_type=instance_type,endpoint_name=endpoint_name)


If you have already deployed your model, you can also access it via your chosen `endpoint_name` and `sagemaker_session`:


In [ ]:
deployed_model = BriaPredictor(endpoint_name=endpoint_name, sagemaker_session=sagemaker_session)

In [ ]:
output = deployed_model.predict(GenerationRequest(text_prompts=[TextPrompt(text="jaguar in the Amazon rainforest")],
                                             # style_preset="cinematic",
                                             seed = 12345,
                                            width=1024,
                                            height=1024.
                                             ))


Output images are included in the response's `artifacts` as base64 encoded strings. Below is a helper function for accessing decoding these images:

In [ ]:
def decode_and_show(model_response: GenerationResponse) -> None:
    """
    Decodes and displays an image from Bria output

    Args:
        model_response (GenerationResponse): The response object from the deployed Bria model.

    Returns:
        None
    """
    image = model_response.artifacts[0].base64
    image_data = base64.b64decode(image.encode())
    image = Image.open(io.BytesIO(image_data))
    display(image)

decode_and_show(output)


In [ ]:
# Here is the original image:
display(Image.open('Cat_August_2010-4.jpg'))

# <a id='toc5_'></a>[3: Delete the endpoint](#toc0_)

When you've finished working, you can delete the endpoint to release the EC2 instance(s) associated with it, and stop billing.

Get your list of Sagemaker endpoints using the AWS Sagemaker CLI like this:

In [ ]:
!aws sagemaker list-endpoints

# Delete an endpoint

In [ ]:
deployed_model.sagemaker_session.delete_endpoint(endpoint_name)
# Rerun the aws cli command above to confirm that its gone.